In [ ]:
import os
import ray

ray.init(num_cpus=8, num_gpus=1)

# @ray.remote(num_gpus=1)
# def use_gpu():
#     print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()))
#     print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    
def objective(seq_len, hidden_dim, learning_rate):
    mse_testing = lstm_model(seq_len, hidden_dim, learning_rate)
    return mse_testing


def training_function(config):
    # Hyperparameters
    seq_len, hidden_dim, learning_rate = config["seq_len"], config["hidden_dim"], config["learning_rate"]
    for i in range(5):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(seq_len, hidden_dim, learning_rate)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)

analysis = tune.run(
    training_function, 
    config={
        "hidden_dim": tune.grid_search([8, 16, 32]),
        "learning_rate": tune.grid_search([0.01,0.01, 0.001]),
        "seq_len": tune.grid_search([5, 10, 15])})

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df